
Need *GaussOutput* (energy & dipoles) & *GaussInput* (geometries) folders 

Upload *zip* & extract before proceed

Fix function-call cell to get state *(0 = S1, 1 = S2)* , numstates = 2 pre-setted

In [ ]:
numstates=2 
def extract_prop(filePath,numstates=2):
    #return list of properties (excited-state energy, oscillator strength)
    import re
    properties = []    
    with open(filePath,'r') as f:
        count = 0
        dipole1 = None
        dipole2 = None
        for line in f:
            if (count >= numstates): 
                break
            match0 = re.match(r"^.*Ground.*to.*excited.*state.*transition.*electric.*dipole.*moments.*\(Au\):", line) # https://regex101.com/r/6Pk3E7/1
            match = re.match(r"^ Excited State.*\d:.*Singlet-A *(\d+.\d+).*eV.*\d+.\d+ nm.*f=(\d+.\d+).*<S\*\*2>=*\d+.\d+", line) # https://regex101.com/r/NILYY6/1         
            if match0:    
                next(f)
                dipoleLine1 = next(f)
                matchD1 = re.match("^ *1 *[-+]?\d+.\d+.*[-+]?\d+.\d+.*[-+]?\d+.\d+\s+[-+]?(\d+.\d+)\s+[-+]?\d+.\d+", dipoleLine1)
                dipole1 = matchD1.group(1)
                # print(dipole1)
                dipoleLine2 = next(f)
                matchD2 = re.match("^ *2 *[-+]?\d+.\d+\s+[-+]?\d+.\d+\s+[-+]?\d+.\d+\s+[-+]?(\d+.\d+) .*\d+.\d+", dipoleLine2)
                # print(dipoleLine2)
                dipole2 = matchD2.group(1)
                # print(dipole2)
            if match:
                energy = match.group(1)
                oscil_strength = match.group(2)
                if (count==0):  properties.append((energy, oscil_strength,dipole1))
                if (count==1):  properties.append((energy, oscil_strength,dipole2))
                count = count + 1

    properties.append(filePath)
    return properties 

# filePath1 = os.path.join(".","GaussOutput","geom_104.log")
# prop = extract_prop(filePath1)
# print(prop)

# -------------------------- generate E & transdipole moment -----------------------------
def genSchNetInput(property_dir="GaussOutput", geom_dir="GaussInput", state=0):
    '''
      take in file (Output) 
      find matching geom (Input) 
      extract prop from Output, grab geom from Input 
      write to xyz output 1. Energy & dipole , 2. geom
    '''
    molecule="indole"
    method="CAM-B3LYP_6-31G_d"
    numstates = 2 
    startLine = 6 # where indole xyz coords start in Gauss .inp file
    # -------------------- open file to write --------------------
    fileWriteName = os.path.join(".", "SchNetInput","energy_dipole_S"+str(state+1)+"_"+molecule+"_"+method+".xyz")
    os.makedirs(os.path.dirname(fileWriteName), exist_ok=True) # create folder if not existed yet
    fileWriter = open(fileWriteName,"w+")

    # ----------------------------- scan for matching geoms in/output files --------------------------
    for f in os.listdir(property_dir): # loop output
        filePath1 = os.path.join(".",property_dir,f)
        geom = (os.path.splitext(os.path.basename(f))[0])
        for f2 in os.listdir(geom_dir): # loop input
            input = (os.path.splitext(os.path.basename(f2))[0])
            # --------------write to xyz--------------
            if(input==geom):
                # ------------------- extract prop from f, geom from f2 -----------------
                # read num atoms of INDOLE
                fileWriter.write(str(16)+"\n")
                # read prop over
                prop = extract_prop(filePath1, numstates=numstates)
                delim = ", "
                print(str(prop))
                # energy1, os1, dp1 = prop[state]
                # energy2, os2 = prop[state+1]

                # convert--------------------------------------------------------------------------------

                # turn result to string-------------------------------------------------------------------------------
                res = delim.join(map(str, prop[state]))
                fileWriter.write(str(res)+"\n")
                
                # read INDOLE over
                # skip first 7 lines 
                # read till end
                filePath2 = os.path.join(".",geom_dir,f2)
                fileReader2 = open(filePath2,"r",encoding='utf-8')
                line = fileReader2.readline()
                for _ in range(startLine): # skip first startLine lines
                    next(fileReader2)
                while (line != ''):
                    line = fileReader2.readline()
                    # first = line.split()[0]
                    # line = line.replace(first,first[0])
                    fileWriter.write(line)
                fileReader2.close()
    
    fileWriter.close()

Function-call cell, fix here *(0 = S1, 1 = S2)*

In [ ]:
genSchNetInput(state=0) # build for each state 0 / 1 (0 = S1, 1 = S2)

Generate .db file for model 

In [ ]:
# --------------------------------- Generate .db file for model ------------------------------- 
'''
change name energy_dipole_<state>_indole_CAM-B3LYP_6-31G_d.xyz 
& 
SchNet_energy_dipole_<state>.db
'''
from schnetpack.data.atoms import AtomsData
from ase.io import read
import numpy as np
import schnetpack as spk
import os

# comment line is weirdly stored in the info dictionary as key by ASE. here it corresponds to the energy
atoms = read(os.path.join("SchNetInput","energy_dipole_S1_indole_CAM-B3LYP_6-31G_d.xyz"), index=':')
print(len(atoms))
print('Energy:', atoms[0].info)
print()
properties = ['energy', 'oscil_strength', 'dipole_moment']

# %rm './SchNet_S2.db'
new_dataset = AtomsData('./SchNet_energy_dipole_S1.db', available_properties=properties)

# parse properties as list of dictionaries
# energy_list = []
# dipole_moment]
for at in atoms:
    # All properties need to be stored as numpy arrays.
    # Note: The shape for scalars should be (1,), not ()
    # Note: GPUs work best with float32 data
    energy = np.array([float(list(at.info.keys())[0].replace(',', ''))], dtype=np.float32)
    oscil_strength = np.array([float(list(at.info.keys())[1].replace(',', ''))], dtype=np.float32)
    dipole_moment = np.array([float(list(at.info.keys())[2])], dtype=np.float32)

    new_dataset.add_system(at,energy=energy, oscil_strength=oscil_strength, dipole_moment=dipole_moment)

Result check

In [ ]:
# ----------------------- check result --------------------------
print('Number of reference calculations:', len(new_dataset))
print('Available properties:')

for p in new_dataset.available_properties:
    print('-', p)
print()

example = new_dataset[0]
print('Properties of molecule with id 0:')

for k, v in example.items():
    print('-', k, ':', v.shape)